In [11]:
import os, math, numpy as np

In [12]:
def get_distance(source: tuple, destination : tuple) -> float:
    return ((source[1] - destination[1]) ** 2 + (source[0] - destination[0]) ** 2)**0.5

In [13]:
### Calculate travel time between two points: origin and destination
def traveltime(origin_id,destination_id,meters_per_minute,locations):
    dist=np.sqrt((locations.at[destination_id,'x']-locations.at[origin_id,'x'])**2\
                +(locations.at[destination_id,'y']-locations.at[origin_id,'y'])**2)
    tt=np.ceil(dist/meters_per_minute)
    return tt

# Read in data instance
A folder of data instance has 5 files: (1) couriers.txt, (2) instance_characteristics.txt, (3) instance_parameters.txt, (4) orders.txt, (5) restaurants.txt

In [14]:
# Read instance information
def read_instance_information(instance_dir):
    orders=pd.read_table(os.path.join(instance_dir,'orders.txt'))
    restaurants=pd.read_table(os.path.join(instance_dir,'restaurants.txt'))
    couriers=pd.read_table(os.path.join(instance_dir,'couriers.txt'))
    instanceparams=pd.read_table(os.path.join(instance_dir,'instance_parameters.txt'))

    order_locations=pd.DataFrame(data=[orders.order,orders.x,orders.y]).transpose()
    order_locations.columns=['id','x','y']
    restaurant_locations=pd.DataFrame(data=[restaurants.restaurant,restaurants.x,restaurants.y]).transpose()
    restaurant_locations.columns=['id','x','y']
    courier_locations=pd.DataFrame(data=[couriers.courier,couriers.x,couriers.y]).transpose()
    courier_locations.columns=['id','x','y']
    locations=pd.concat([order_locations,restaurant_locations,courier_locations])
    # locations.set_index('id',inplace=True)
    #
    # orders.set_index('order',inplace=True)
    # couriers.set_index('courier',inplace=True)
    # restaurants.set_index('restaurant',inplace=True)

    meters_per_minute=instanceparams.at[0,'meters_per_minute']
    pickup_service_minutes=instanceparams.at[0,'pickup service minutes']
    dropoff_service_minutes=instanceparams.at[0,'dropoff service minutes']
    target_click_to_door=instanceparams.at[0,'target click-to-door']
    pay_per_order=instanceparams.at[0,'pay per order']
    guaranteed_pay_per_hour=instanceparams.at[0,'guaranteed pay per hour']

    return orders,restaurants,couriers,instanceparams,locations

In [15]:
import pandas as pd
instance_dir = 'data/0o50t75s1p100'
orders,restaurants,couriers,instanceparams,locations=read_instance_information(instance_dir)

# Set hyper-parameters:
f∶ every f minutes solves a matching problem to prescribe the next pick-up and delivery assignment for each courier<br>
t : optimization time<br>
∆_(u): t+ ∆_(u ) is the assignment horizon<br>
∆_(1); ∆_(2): to determine Z_(t)<br>
beta: control the freshness in the construction of bundles
gamma: control the click to door time in the construction of bundles


In [16]:
# Accorting to default values in the paper
f = 5
delta_u = 10
delta_1 = 10
delta_2 = 10
# Not find in the paper
beta = 10 # should be tuned
gamma = 10 # should be tuned

# Procedure to initialize a solution

**Function that return U(t,r)**<br>
U(t,r) is the set of upcoming orders at restaurant r

**Courier Object**

In [31]:
class Order:
    def __init__(self, order_information : dict):

        self.order_id = order_information.get('order')
        self.destination = (order_information.get('x'), order_information.get('y'))
        self.placement_time = order_information.get('placement_time')
        self.restaurant_id = order_information.get('restaurant')
        self.ready_time = order_information.get('ready_time')

    def __repr__(self):
        return "order_id : {} destination : {} placement_time : {} restaurant_id : {} ready_time : {}".format(
            self.order_id, self.destination, self.placement_time, self.restaurant_id, self.ready_time
        )


In [32]:
class Courier(object):
    def __init__(self, courier_information : dict, restaurant : dict):
        self.courier = courier_information.get('courier')
        self.x = courier_information.get('x')
        self.y = courier_information.get('y')
        self.on_time = courier_information.get('on_time')
        self.off_time = courier_information.get('off_time')

        # Update and derived along the way
        self.travel = []
        self.visited = []
        self.orders = []
        # self.next_available_time = # a dictionary containing restaurant as key and available time as value
        self.current_position = (self.x, self.y)
        self.current_restaurant = None
        self.distance_travelled = 0
        self.current_time = self.on_time
        self.restaurant = restaurant

    @staticmethod
    def calculate_travel_time(source: tuple,
                              restaurant:tuple,
                              destination: tuple,
                              meters_per_minute : float) -> float:

        return get_distance(source, restaurant) / meters_per_minute + get_distance(restaurant, destination) / meters_per_minute

   # check if the courier can take the order
    def can_assign(self, order : Order) -> bool:
        # if the ready time < courier's off time
        if order.ready_time < self.on_time:
            return False
        if order.ready_time > self.off_time:
            return False

    # assign an order to courier
    def assign_order(self, order: Order) -> bool:
        # update available time for the next assignment
        _, restaurant_x, restaurant_y = restaurants[restaurants['restaurant'] == order.restaurant].values[0]
        if self.can_assign(order):
            self.orders.append(order)
            self.current_time += self.calculate_travel_time(self.current_position, (restaurant_x, restaurant_y), orders.destination)

    def __repr__(self):
        return "courier_id : {} current_coord : {} on_time : {} off_time : {} current_time : {}".format(
            self.courier, (self.x, self.y), self.on_time, self.off_time, self.current_time
        )

In [36]:
class DeliveryRouting:
    def __init__(self, instance_dir : str, target_bundle_size : int = 5, f : int = 5, delta_u : int = 10):
        self.target_bundle_size = target_bundle_size
        self.delta_u = delta_u

        orders, restaurants, couriers, instanceparams, locations = read_instance_information(instance_dir)

        self.orders = [Order(order) for order in orders.to_dict(orient = 'records')]
        self.orders = sorted(self.orders, key = lambda x: x.order_id)

        self.restaurants = restaurants.to_dict(orient = 'records')
        self.couriers = [Courier(courier, self.restaurants) for courier in couriers.to_dict(orient = 'records')]
        self.unassigned_orders = self.orders.copy()

        self.instance_params = instanceparams.to_dict(orient = 'records')[0]

        self.meters_per_minute, self.pickup_service_minutes, self.dropoff_service_minutes, \
            self.target_click_to_door, self.maximum_click_to_door, self.pay_per_order,\
            self.guaranteed_pay_per_hour = self.instance_params.values()

    def calculate_travel_time(self,
                              source: tuple,
                              restaurant:tuple,
                              destination: tuple) -> float:

        meters_per_minute = self.meters_per_minute
        return get_distance(source, restaurant) / meters_per_minute + get_distance(restaurant, destination) / meters_per_minute

    def can_arrive(self,
                   source : tuple,
                   restaurant : tuple,
                   destination : tuple,
                   current_time : float,
                   ready_time : float):

        travel_time = self.calculate_travel_time(source, restaurant, destination)

        if current_time + travel_time <= ready_time:
            return False
        return True

    def get_bundle_size(self, z : int) -> int :
        # available = [courier for courier in self.couriers if courier.]
        pass

    def initial_bundle(self, t):
        bundles = {}

        for order in self.orders:
            if order.ready_time <= t + self.delta_u and order.ready_time >= t:
                if order.restaurant_id not in bundles():
                    bundles[order.restaurant_id] = [order]
                else:
                    bundles[order.restaurant_id].append(order)

        return bundles

In [37]:
instance_dir = 'data/0o50t75s1p100'
dr = DeliveryRouting(instance_dir)

In [35]:
dr.unassigned_orders

[order_id : o1 destination : (8317, 5587) placement_time : 743 restaurant_id : r1 ready_time : 753,
 order_id : o10 destination : (8918, 5918) placement_time : 563 restaurant_id : r9 ready_time : 593,
 order_id : o100 destination : (2116, 3455) placement_time : 540 restaurant_id : r39 ready_time : 544,
 order_id : o101 destination : (7985, 1932) placement_time : 635 restaurant_id : r53 ready_time : 655,
 order_id : o102 destination : (136, 2824) placement_time : 515 restaurant_id : r11 ready_time : 537,
 order_id : o103 destination : (5828, 3746) placement_time : 515 restaurant_id : r54 ready_time : 543,
 order_id : o104 destination : (5730, 5275) placement_time : 496 restaurant_id : r13 ready_time : 508,
 order_id : o105 destination : (10215, 4855) placement_time : 263 restaurant_id : r27 ready_time : 274,
 order_id : o106 destination : (8674, 7518) placement_time : 665 restaurant_id : r39 ready_time : 673,
 order_id : o107 destination : (7808, 7411) placement_time : 528 restaurant_id

**Funtion that calculate bundel size Z(t)**

In [ ]:
def get_bundel_size():
    ...

**Function that calculate k(r,t)**<br>
k(r,t) is the number of couriers who are available at restaurant r

In [ ]:
def get_available_courier(t,r):
    ...

**Route object**<br>
A route is a bundle,i.e: a list of ordered orders

In [ ]:
class Route(object):
    def __init__:
    
    # calculate total travel time from 1st destination to the last destination of the route
    def total_travel_time():

    # calculate total service delay    
    def total_service_delay():
        #<to be discuss>

    # calculate total_click_to_door 
    def total_click_to_door():
        #<to be discuss>

    # calculate route efficiency: travel time per order:
    def route_efficiency():

    # calculate route cost
    def route_cost():
        #route_cost = total_travel_time + beta*total_service_delay + gamma*total_click_to_door


# Implement procedure 1